# Challenge Data Science - Setembro 2022

**Identificar pacientes com alto risco cardiovascular**

Autor: Jorge Chamby Diaz
_________

## O Problema

As doenças cardiovasculares (DCVs) são a causa número 1 de morte em todo o mundo, levando cerca de 17,9 milhões de vidas a cada ano, o que representa 31% de todas as mortes em todo o mundo. Quatro de cada 5 mortes por CVDs são devido a ataques cardíacos e derrames, e um terço dessas mortes ocorre prematuramente em pessoas com menos de 70 anos de idade. 

Pessoas com doença cardiovascular ou com alto risco cardiovascular (devido à presença de um ou mais fatores de risco, como hipertensão, diabetes, hiperlipidemia ou doença já estabelecida) precisam de detecção e gerenciamento precoces, onde um modelo de aprendizado de máquina pode ser de grande ajuda.

## Os dados

A insuficiência cardíaca é um evento comum causado por DCVs e este conjunto de dados contém 11 características que podem ser usadas para prever uma possível doença cardíaca:

- Medidas de 11 variáveis que caracterizam cada amostra (as features do problema):
<br><br>
    - 1 - Age: idade do paciente (anos)
    - 2 - Sex: sexo do paciente (M: Masculino, F: Feminino)
    - 3 - ChestPainType: tipo de dor no peito (TA: Angina Típica, ATA: Angina Atípica, NAP: Dor Não Anginosa, ASY: Assintomática)
    - 4 - RestingBP: pressão arterial de repouso (mm Hg)
    - 5 - Cholesterol: colesterol sérico (mm/dl)
    - 6 - FastingBS: glicemia em jejum (1: se FastingBS > 120 mg/dl, 0: caso contrário)
    - 7 - RestingECG: resultados do eletrocardiograma de repouso (Normal: normal, ST: com anormalidade da onda ST-T, LVH: mostrando provável ou definitiva hipertrofia ventricular esquerda pelos critérios de Estes)
    - 8 - MaxHR: frequência cardíaca máxima alcançada (Valor numérico entre 60 e 202)
    - 9 - ExerciseAngina: angina induzida por exercício (Y: Sim, N: Não)
    - 10 - Oldpeak: pico antigo = ST (Valor numérico medido em depressão)
    - 11 - ST_Slope: a inclinação do segmento ST do exercício de pico (Up: upsloping, Flat: flat, Down: downsloping)    
<br><br>
- Além disso, há a variável resposta que no caso é uma variável binária:
<br><br>
    - 12 - HeartDisease: classe de saída (1: doença cardíaca, 0: normal)

Para maiores informações sobre a coleta e origem dos dados, veja a página do dataset no repositório UCI machine learning repository, [disponível aqui!](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/) 

**Para baixar os conjuntos de dados, podem acessar diretamente a este [link](https://github.com/jchambyd/Datasets/tree/main/Heart-disease)**

## O Challenge

Suponha que você é um cientista de dados que trabalha na área de *bioinformatica* de uma clínica especializada no tratamento de doenças cardiovasculares. Nos últimos meses a empresa tem recebido uma grande quantidade de pacientes (o que é muito bom!), permitindo gerar um histórico de todos os pacientes que foram para consulta e saíram com um diagnóstico. Para simplificar as coisas, o diagnóstico foi reduzido a duas possíveis respostas: 'positivo' ou 'negativo'. Positivo significa que o paciente apresenta alguma doença cardiovascular, e negativo significa que a pessoa está livre de doenças cardiovasculares. A empresa pede para você construir um modelo preditivo que permita, a partir de um conjunto prévio de exames feitos sobre um paciente novo, identificar se o paciente possui ou não uma doença cardiovascular.

Com este modelo, os médicos vão ter uma ferramenta de apoio muito útil para realizar o diagnóstico de doenças cardiovasculares para novos pacientes que a clínica receba no futuro.

Dentro deste contexto, seu objetivo como cientista de dados é claro:

> Agregar valor ao negócio, explorando os dados que você tem à disposição.

Na primeira sprint do projeto, você e outros colegas do time de ciência de dados chegaram na seguinte _TO-DO list_ para o projeto. Algumas atividades já foram executadas nas primeiras semanas de trabalho (o time é muito bom!), mas agora ainda restam algumas atividades muito importantes (e divertidas!) a serem feitas:

- [x] Ingestão dos dados e detalhada análise exploratória
- [x] Formulação do problema
- [ ] Primeiro modelo baseline
- [ ] Iterações pelo ciclo de modelagem
- [ ] Compílação dos resultados para o negócio
- [ ] Comunicação dos resultados

Bom trabalho, e divirta-se! :D

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, make_scorer, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
import xgboost as xgb

In [15]:
heart_data = pd.read_csv('./datasets/heart.csv')

In [16]:
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [17]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [18]:
heart_data_copy = heart_data.copy()
y = heart_data_copy['HeartDisease']

# preparando os dados do modelo

x = heart_data_copy.drop(columns = 'HeartDisease')

x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.3, stratify=y)

cat_features = x_train.select_dtypes(exclude=np.number).columns.tolist()

pipe_cat = Pipeline([('ordinal_encoder', OrdinalEncoder())])

preprocessing = ColumnTransformer([('cat_transform', pipe_cat, cat_features)],remainder='passthrough')



In [19]:
# preparando o pipeline do modelo
model_pipe = Pipeline([('preprocessing', preprocessing),
                      ('std_scaler', StandardScaler()),
                      ('xgboost', xgb.XGBClassifier(booster= 'gbtree'))],
                      )

In [20]:
# dicionário de parâmetro
param_dict = {'xgboost__eta': np.linspace(0,1,100),
              'xgboost__max_depth': np.random.randint(10,50,500)}

In [21]:
# métricas de avaliação
metrics = {'precision_score': make_scorer(precision_score),
           'accuracy_score': make_scorer(accuracy_score),
           'f1_score': make_scorer(f1_score, pos_label = 1),
           'recall_score': make_scorer(recall_score, pos_label = 1)}

In [22]:
kf5 = StratifiedKFold(n_splits=5, shuffle=True)

In [23]:
model = RandomizedSearchCV(estimator=model_pipe, param_distributions=param_dict, scoring=metrics, n_jobs=-1,cv=kf5, refit='recall_score')

In [24]:
model.fit(x_train,y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('cat_transform',
                                                                               Pipeline(steps=[('ordinal_encoder',
                                                                                                OrdinalEncoder())]),
                                                                               ['Sex',
                                                                                'ChestPainType',
                                                                                'RestingECG',
                                                                                'ExerciseAngina',
                                                            

In [25]:
model.best_params_

{'xgboost__max_depth': 28, 'xgboost__eta': 0.9595959595959597}

In [26]:
y_predict = model.best_estimator_.predict(x_test)

print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       287
           1       0.88      0.83      0.86       356

    accuracy                           0.84       643
   macro avg       0.84      0.85      0.84       643
weighted avg       0.85      0.84      0.84       643

